# Load packages and libraries

In [22]:
.libPaths()
assign(".lib.loc", "/home/manuel.tardaguila/conda_envs/multiome_NEW_downstream_analysis/lib/R/library", envir = environment(.libPaths))
.libPaths()
# sessionInfo()


suppressMessages(library(Seurat))
suppressMessages(library(Signac))
suppressMessages(library(dplyr)) 
suppressMessages(library(ggplot2)) 
suppressMessages(library(Matrix)) 
suppressMessages(library(data.table)) 
suppressMessages(library(ggpubr)) 
suppressMessages(library(ggplot2))
suppressMessages(library(pheatmap))
suppressMessages(library(presto))
suppressMessages(library("qlcMatrix"))
suppressMessages(library("cowplot"))
suppressMessages(library("RColorBrewer"))
suppressMessages(library("plyr"))
suppressMessages(library("forcats"))
suppressMessages(library('ggeasy'))
suppressMessages(library('dplyr'))
suppressMessages(library("svglite"))
suppressMessages(library("ape"))
suppressMessages(library("ggforce"))
suppressMessages(library("tidyr"))
suppressMessages(library("edgeR"))
suppressMessages(library("apeglm"))
suppressMessages(library("DESeq2"))
suppressMessages(library("tibble")) 
suppressMessages(library(future))

library("optparse")
suppressMessages(library("splitstackshape")) 
suppressMessages(library("ggupset"))


[1] "/home/manuel.tardaguila/conda_envs/multiome_NEW_downstream_analysis/lib/R/library"

[1] "/home/manuel.tardaguila/conda_envs/multiome_NEW_downstream_analysis/lib/R/library"

# Read in miRNA counts

In [4]:
setwd("/group/soranzo/manuel.tardaguila/small_rna/new_trimming/mirna_quant/edger_qc")

miRNA_counts <- read.csv("mature_counts.csv")

row.names(miRNA_counts)<-miRNA_counts$X
cat("miRNA_counts/n")
#cat(str(miRNA_counts))
cat("/n")

miRNA_counts/n/n

## Transform into a DESeq2 count matrix

In [5]:
miRNA_counts_transposed <- t(miRNA_counts)

cat("miRNA_counts_transposed/n")
cat(str(miRNA_counts_transposed))
cat("/n")

miRNA_counts_transposed/n chr [1:2382, 1:6] "wt_1" " 6" "0" "2" " 1" "2" "13" "0" " 54" "32" "1" "0" ...
 - attr(*, "dimnames")=List of 2
  ..$ : chr [1:2382] "X" "hsa.let.7a.5p" "hsa.let.7a.3p" "hsa.let.7a.2.3p" ...
  ..$ : chr [1:6] "wt_1" "rs62237617_1" "wt_2" "rs62237617_2" ...
/n

In [8]:
miRNA_counts_transposed.matrix<-as.matrix(miRNA_counts_transposed[-which(row.names(miRNA_counts_transposed) == 'X'),])

cat("miRNA_counts_transposed.matrix/n")
cat(str(miRNA_counts_transposed.matrix))
cat("/n")

miRNA_counts_transposed.matrix/n chr [1:2381, 1:6] " 6" "0" "2" " 1" "2" "13" "0" " 54" "32" "1" "0" "2" ...
 - attr(*, "dimnames")=List of 2
  ..$ : chr [1:2381] "hsa.let.7a.5p" "hsa.let.7a.3p" "hsa.let.7a.2.3p" "hsa.let.7b.5p" ...
  ..$ : chr [1:6] "wt_1" "rs62237617_1" "wt_2" "rs62237617_2" ...
/n

In [36]:
count_matrix<-apply(miRNA_counts_transposed.matrix, 2, as.numeric)

row.names(count_matrix)<-row.names(miRNA_counts_transposed.matrix)

cat("count_matrix/n")
cat(str(count_matrix))
cat("/n")

count_matrix/n num [1:2381, 1:6] 6 0 2 1 2 13 0 54 32 1 ...
 - attr(*, "dimnames")=List of 2
  ..$ : chr [1:2381] "hsa.let.7a.5p" "hsa.let.7a.3p" "hsa.let.7a.2.3p" "hsa.let.7b.5p" ...
  ..$ : chr [1:6] "wt_1" "rs62237617_1" "wt_2" "rs62237617_2" ...
/n

In [37]:
count_matrix <- round(count_matrix)
storage.mode(count_matrix) <- "integer"

In [38]:
# Check the data type
print(class(count_matrix))

# Check the minimum value
min_count <- min(count_matrix)
print(paste("Minimum value in matrix:", min_count))

# Check for negative values
if (min_count < 0) {
  # Identify how many negative values exist
  num_negative <- sum(count_matrix < 0)
  print(paste("Number of negative values:", num_negative))

  # Print a few examples of genes with negative counts (optional)
  negative_rows <- head(which(rowSums(count_matrix < 0) > 0))
  print("Rows containing negative values (first few):")
  print(rownames(count_matrix)[negative_rows])
}

[1] "matrix" "array" 
[1] "Minimum value in matrix: 0"


## check presence  of mir5739

In [84]:
row.names(count_matrix)[grep("5739", row.names(count_matrix))]

[1] "hsa.miR.5739"

In [85]:
count_matrix[which(row.names(count_matrix) == 'hsa.miR.5739'),]

wt_1 rs62237617_1         wt_2 rs62237617_2         wt_3 rs62237617_3 
          11           15           14            4            5            6

# Create the colData object

In [15]:
samples<-colnames(miRNA_counts_transposed.matrix)

cat("samples/n")
cat(str(samples))


samples/n chr [1:6] "wt_1" "rs62237617_1" "wt_2" "rs62237617_2" "wt_3" ...


In [16]:
condition<-gsub("_.+$","",samples)

cat("condition/n")
cat(str(condition))


condition/n chr [1:6] "wt" "rs62237617" "wt" "rs62237617" "wt" "rs62237617"


In [17]:
batch<-gsub("^[^_]+_","",samples)

cat("batch/n")
cat(str(batch))


batch/n chr [1:6] "1" "1" "2" "2" "3" "3"


In [18]:
colData<-as.data.frame(cbind(samples,condition,batch))

colnames(colData)<-c('sample','condition','batch')

cat("colData_0/n")
cat(str(colData))


colData/n'data.frame':	6 obs. of  3 variables:
 $ sample   : chr  "wt_1" "rs62237617_1" "wt_2" "rs62237617_2" ...
 $ condition: chr  "wt" "rs62237617" "wt" "rs62237617" ...
 $ batch    : chr  "1" "1" "2" "2" ...


## factorize condition

In [19]:
colData$condition<-factor(colData$condition)
colData$condition<-relevel(colData$condition, ref='wt') ### Nothing works with ordered factors

colData$condition<-factor(colData$condition)
colData$condition<-relevel(colData$condition, ref='wt') ### Nothing works with ordered factors


cat("colData_1/n")
cat(str(colData))

colData_1/n'data.frame':	6 obs. of  3 variables:
 $ sample   : chr  "wt_1" "rs62237617_1" "wt_2" "rs62237617_2" ...
 $ condition: Factor w/ 2 levels "wt","rs62237617": 1 2 1 2 1 2
 $ batch    : chr  "1" "1" "2" "2" ...


# DESeq2

## Create the DESeqDataSetFromMatrix object

In [39]:
dds <- DESeqDataSetFromMatrix(countData = count_matrix,
                              colData = colData,
                              design = ~ condition)

In [41]:
cat("dds_0/n")
#cat(str(dds))

dds_0/n

## Filter out miRNAs with fewer than  total raw counts

In [50]:
keep <- rowSums(counts(dds)) >= 10

filtered_out<-keep[which(keep == "FALSE")]

cat(str(filtered_out))

 Named logi [1:1189] FALSE FALSE FALSE FALSE FALSE FALSE ...
 - attr(*, "names")= chr [1:1189] "hsa.let.7a.3p" "hsa.let.7d.5p" "hsa.let.7e.3p" "hsa.let.7f.5p" ...


In [51]:
grep("5739",filtered_out)

integer(0)

In [52]:
dds <- dds[keep,]

## estimateSizeFactors----------------------------------


In [55]:
dds<-estimateSizeFactors(dds)

## Run the rest

In [56]:
dds <- DESeq(dds)

using pre-existing size factors

estimating dispersions

found already estimated dispersions, replacing these

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



## Extract results

In [58]:
possible_contrasts<-colnames(dds@modelMatrix)[-1] # -1 because 1 is the Intercept term

possible_contrasts

[1] "condition_rs62237617_vs_wt"

In [57]:
resultsNames(dds)

[1] "Intercept"                  "condition_rs62237617_vs_wt"

In [63]:
tmp_results<-results(dds, test= "Wald", name=possible_contrasts[1], independentFiltering=FALSE)
        
#### expand LogFC #########################################

tmp_results <- lfcShrink(dds, 
                         coef = possible_contrasts[1],
                         res=tmp_results,
                         type = "apeglm")


cat("tmp_results_0/n")
#cat(str(tmp_results))

using 'apeglm' for LFC shrinkage. If used in published research, please cite:
    Zhu, A., Ibrahim, J.G., Love, M.I. (2018) Heavy-tailed prior distributions for
    sequence count data: removing the noise and preserving large differences.
    Bioinformatics. https://doi.org/10.1093/bioinformatics/bty895



tmp_results_0/n

In [66]:
 #### obtain data frame #########################################
        
tmp_tb <- as.data.frame(tmp_results %>%
                          data.frame() %>%
                          rownames_to_column(var = "miRNA") %>%
                          as_tibble() %>%
                          arrange(padj), stringsAsFactors=F)

tmp_tb$contrast<-possible_contrasts[1]

cat("tmp_tb_0")
cat(str(tmp_tb))

tmp_tb_0/n'data.frame':	1192 obs. of  7 variables:
 $ miRNA         : chr  "hsa.let.7a.5p" "hsa.let.7a.2.3p" "hsa.let.7b.5p" "hsa.let.7b.3p" ...
 $ baseMean      : num  15.27 2.54 3.81 2.7 58.22 ...
 $ log2FoldChange: num  1.61e-06 1.91e-07 1.34e-06 1.12e-06 -1.30e-06 ...
 $ lfcSE         : num  0.00144 0.00144 0.00144 0.00144 0.00144 ...
 $ pvalue        : num  0.2882 0.7964 0.1077 0.0794 0.6092 ...
 $ padj          : num  0.999 0.999 0.999 0.999 0.999 ...
 $ contrast      : chr  "condition_rs62237617_vs_wt" "condition_rs62237617_vs_wt" "condition_rs62237617_vs_wt" "condition_rs62237617_vs_wt" ...


In [67]:
tmp_tb$minuslog10padj <- -log10(tmp_tb$padj)
      
tmp_tb$abslogfc<-abs(tmp_tb$log2FoldChange)

cat("tmp_tb_0")
cat(str(tmp_tb))

tmp_tb_0'data.frame':	1192 obs. of  9 variables:
 $ miRNA         : chr  "hsa.let.7a.5p" "hsa.let.7a.2.3p" "hsa.let.7b.5p" "hsa.let.7b.3p" ...
 $ baseMean      : num  15.27 2.54 3.81 2.7 58.22 ...
 $ log2FoldChange: num  1.61e-06 1.91e-07 1.34e-06 1.12e-06 -1.30e-06 ...
 $ lfcSE         : num  0.00144 0.00144 0.00144 0.00144 0.00144 ...
 $ pvalue        : num  0.2882 0.7964 0.1077 0.0794 0.6092 ...
 $ padj          : num  0.999 0.999 0.999 0.999 0.999 ...
 $ contrast      : chr  "condition_rs62237617_vs_wt" "condition_rs62237617_vs_wt" "condition_rs62237617_vs_wt" "condition_rs62237617_vs_wt" ...
 $ minuslog10padj: num  0.000395 0.000395 0.000395 0.000395 0.000395 ...
 $ abslogfc      : num  1.61e-06 1.91e-07 1.34e-06 1.12e-06 1.30e-06 ...


In [68]:
DE_results<-tmp_tb

## Extract normalized counts

In [74]:
nor_counts<-as.matrix(counts(dds, normalized=TRUE))
      
row.names(nor_counts)<-row.names(nor_counts)

cat("nor_counts_0\n")
str(nor_counts)
cat("\n")


nor_counts_0
 num [1:1192, 1:6] 6.28 2.09 1.05 2.09 13.6 ...
 - attr(*, "dimnames")=List of 2
  ..$ : chr [1:1192] "hsa.let.7a.5p" "hsa.let.7a.2.3p" "hsa.let.7b.5p" "hsa.let.7b.3p" ...
  ..$ : chr [1:6] "wt_1" "rs62237617_1" "wt_2" "rs62237617_2" ...



# SAVE results

In [76]:
setwd("/group/soranzo/manuel.tardaguila/small_rna/new_trimming/")

In [77]:
write.table(DE_results, file="DE_results.tsv", sep="\t", quote=F, row.names=F)

write.table(nor_counts, file="nor_counts.tsv", sep="\t", quote=F)


## Explore results

In [82]:


test<-DE_results[which(DE_results$pvalue <= 0.05),]

cat("test_0\n")
str(test)
cat("\n")

test_0
'data.frame':	17 obs. of  9 variables:
 $ miRNA         : chr  "hsa.miR.107" "hsa.miR.130b.5p" "hsa.miR.769.5p" "hsa.miR.891b" ...
 $ baseMean      : num  6.27 5.23 26.46 1.86 208.14 ...
 $ log2FoldChange: num  -2.17e-06 -1.67e-06 1.72 9.53e-07 -4.95e-06 ...
 $ lfcSE         : num  0.00144 0.00144 0.96248 0.00144 0.00144 ...
 $ pvalue        : num  0.0289 0.035 0.0015 0.025 0.0203 ...
 $ padj          : num  0.999 0.999 0.999 0.999 0.999 ...
 $ contrast      : chr  "condition_rs62237617_vs_wt" "condition_rs62237617_vs_wt" "condition_rs62237617_vs_wt" "condition_rs62237617_vs_wt" ...
 $ minuslog10padj: num  0.000395 0.000395 0.000395 0.000395 0.000395 ...
 $ abslogfc      : num  2.17e-06 1.67e-06 1.72 9.53e-07 4.95e-06 ...



In [83]:
SIG<-DE_results[which(DE_results$minuslog10padj >= 1.3),]

cat("SIG_0\n")
str(SIG)
cat("\n")

SIG_0
'data.frame':	0 obs. of  9 variables:
 $ miRNA         : chr 
 $ baseMean      : num 
 $ log2FoldChange: num 
 $ lfcSE         : num 
 $ pvalue        : num 
 $ padj          : num 
 $ contrast      : chr 
 $ minuslog10padj: num 
 $ abslogfc      : num 

